# Generate Gene List for FunGen xQTL Analysis

## Overview
This analysis combines genes from multiple resources related to Alzheimer's Disease (AD) to generate a comprehensive gene list for the FunGen xQTL project. We integrate data from:

1. **FunGen xQTL** - Including three analytical approaches:
   - Single context analysis
   - Multi-context analysis (157 genes)
   - ColocBoost analysis
   
   *For detailed methodology, see: [Mar2025_xQTL_modalities_GWAS_Loci.ipynb](https://github.com/cumc/xqtl-paper/blob/main/landscape_analysis/Mar2025_xQTL_modalities_GWAS_Loci.ipynb)*

2. **ADSP Alzheimer's Disease Variant Catalog (AVC)** - Genes from Table 2 of the ADSP AVC database
   *Source: [NIAGADS GVC Top Hits List](https://adsp.niagads.org/gvc-top-hits-list/)*

3. **FunGen AD PI List** - Curated gene list contributed by Principal Investigators of the FunGen project

4. **Multi-gene analysis results** - From ROSMAP and MSBB datasets
5. **Genome-wide MSBB multi-context results** - Additional contextual analysis


## Setup and Data Loading

In [42]:
#Load required libraries
library(tidyverse)
library(data.table)

#Set paths and load gene reference
gene_ref <- fread('/data/resource/references/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene.region_list')

## Data Source 1: FunGen xQTL Results

In [43]:
# Load FunGen xQTL gene list (single/multi-context and ColocBoost results)
tb1 <- fread('~/project/FunGen_xQTL/2025/Mar/Mar19_2025.193genes.single_multicon_colocboost_metabrain.gene_list.tsv') %>% 
  mutate(data_resource = 'FunGen_xQTL')

# Display sample records
tb1 %>% head()

gene_id,resources,#chr,start,end,gene_name,data_resource
<chr>,<chr>,<int>,<int>,<int>,<chr>,<chr>
ENSG00000117322,"ColocBoost,single_context,MetaBrain",1,207454229,207454230,CR2,FunGen_xQTL
ENSG00000125944,ColocBoost,1,23344335,23344336,HNRNPR,FunGen_xQTL
ENSG00000135766,ColocBoost,1,231422286,231422287,EGLN1,FunGen_xQTL
ENSG00000142798,ColocBoost,1,21937309,21937310,HSPG2,FunGen_xQTL
ENSG00000143321,ColocBoost,1,156766924,156766925,HDGF,FunGen_xQTL
ENSG00000143771,ColocBoost,1,224356857,224356858,CNIH4,FunGen_xQTL


## Data Source 2: ADSP Alzheimer's Variant Catalog

In [4]:
# Load ADSP AVC data from Table 2
tb2_raw <- fread('~/project/FunGen_xQTL/2025/Mar/GVC - table2.csv', header = F)

# Set column names from the second row and remove header rows
colnames(tb2_raw) <- tb2_raw[2,] %>% as.character()
tb2_raw <- tb2_raw[-c(1:2),]

# Display sample records
tb2_raw %>% head()

# Map gene names to gene reference data and add resource identifier
tb2 <- gene_ref %>% 
  filter(gene_name %in% tb2_raw$Gene) %>% 
  mutate(data_resource = 'FunGen_ADSP')

Number,Gene,Source,Location (hg38)
<chr>,<chr>,<chr>,<chr>
1,ABCA7,GVC,"chr19:1,039,997-1,065,572"
2,ABI3,GVC,"chr17:49,210,411-49,223,225"
3,APOE,GVC,"chr19:44,905,791-44,909,393"
4,APP,GVC,"chr21:25,880,550-26,171,128"
5,APBB3,Novikova,"chr5:140,558,268-140,564,781"
6,BIN1,GVC/Novikova,"chr2:127,048,023-127,107,288"


## Data Source 3: Principal Investigator Gene List

In [5]:
# Load PI-curated gene list
tb3_raw <- fread('~/project/FunGen_xQTL/2025/Mar/Josh_list.csv')

# Display sample records
tb3_raw %>% head()

# Map gene names to gene reference data and add resource identifier
tb3 <- gene_ref %>% 
  filter(gene_name %in% tb3_raw$`Gene name`) %>% 
  mutate(data_resource = 'PI_list')

Gene name,Projects or PI interested
<chr>,<chr>
ABCA7,Shulman
ADAM10,Shulman
AKAP9,Shulman
ALDH1A2,Shulman
AP4M1,Shulman
AQP9,Shulman


## Data Source 4: Multi-gene Analysis Results

### Load AD GWAS fine-mapping results from March 25, 2025

In [6]:
ad <- fread('/data/analysis_result/AD_GWAS_finemapping/export/FunGen_xQTL.ADGWAS.Mar25.exported.toploci.bed.gz')

# Filter for significant variants
ad_vars <- ad %>% filter(cs_coverage_0.95 > 0)

In [7]:
# Load multi-gene analysis results from ROSMAP and MSBB datasets
rosmap <- fread('/data/analysis_result/multi_gene/ROSMAP/export/summary/ROSMAP.exported.toploci.bed.gz')
msbb <- fread('/data/analysis_result/multi_gene/MSBB/export/summary/MSBB.exported.toploci.bed.gz')

# Combine datasets and filter for significant results (credible set coverage > 0.95)
multi_gene <- rbind(
  rosmap %>% mutate(resource = 'ROSMAP_multi_gene'), 
  msbb %>% mutate(resource = 'MSBB_multi_gene')
) %>% 
  filter(cs_coverage_0.95 > 0)

# Display sample records
multi_gene %>% head()

#chr,start,end,a1,a2,variant_ID,gene_ID,event_ID,cs_coverage_0.95,cs_coverage_0.95_purity0.5,PIP,conditional_effect,lfsr,context,resource
<int>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,945258,945259,T,TC,chr1:945259:TC:T,chr1_0_7520074,ENSG00000187961,5,5,0.09419357,0.188749238940897,7.28999870208797e-07,ROSMAP_Exc_DeJager_eQTL,ROSMAP_multi_gene
1,954723,954724,A,G,chr1:954724:G:A,chr1_0_7520074,ENSG00000187961,5,5,0.57161741,0.204391161719796,7.28999870208797e-07,ROSMAP_Exc_DeJager_eQTL,ROSMAP_multi_gene
1,965124,965125,C,G,chr1:965125:G:C,chr1_0_7520074,ENSG00000187961,5,5,0.16732354,0.188129574826807,7.28999870208797e-07,ROSMAP_Exc_DeJager_eQTL,ROSMAP_multi_gene
1,965665,965666,A,ACC,chr1:965666:ACC:A,chr1_0_7520074,ENSG00000187961,5,5,0.13827854,0.185975987146749,7.28999870208797e-07,ROSMAP_Exc_DeJager_eQTL,ROSMAP_multi_gene
1,978508,978509,A,G,chr1:978509:G:A,chr1_0_7520074,ENSG00000187642,5,5,0.25836594,0.484054794439528,8.7517581088359e-15,ROSMAP_PCC_DeJager_eQTL,ROSMAP_multi_gene
1,978952,978953,G,C,chr1:978953:C:G,chr1_0_7520074,ENSG00000187642,5,5,0.07840768,0.480242128208934,8.7517581088359e-15,ROSMAP_PCC_DeJager_eQTL,ROSMAP_multi_gene


In [8]:
# Identify multi-gene results that overlap with AD GWAS variants
multi_gene_ad <- multi_gene %>% filter(variant_ID %in% ad_vars$variant_ID)
multi_gene_ad %>% dim()

# Extract gene IDs from the event_ID field and process
tb4 <- multi_gene_ad %>%
  filter(!is.na(event_ID) & event_ID != '') %>% 
  select(event_ID, resource) %>%
  separate_rows(event_ID, sep = ";") %>% 
  distinct()

# Map gene IDs to gene reference data
tb4 <- gene_ref %>% 
  filter(gene_id %in% tb4$event_ID) %>% 
  mutate(resources = 'multi_gene', data_resource = 'FunGen_xQTL')

tb4 %>% head()

[1] 2885   15

#chr,start,end,gene_id,gene_name,resources,data_resource
<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
chr1,207034365,207034366,ENSG00000123836,PFKFB2,multi_gene,FunGen_xQTL
chr1,207052979,207052980,ENSG00000180667,YOD1,multi_gene,FunGen_xQTL
chr1,207321531,207321532,ENSG00000196352,CD55,multi_gene,FunGen_xQTL
chr1,207454229,207454230,ENSG00000117322,CR2,multi_gene,FunGen_xQTL
chr1,207496146,207496147,ENSG00000203710,CR1,multi_gene,FunGen_xQTL
chr2,134718999,134719000,ENSG00000152128,TMEM163,multi_gene,FunGen_xQTL


## Data Source 5: Genome-Wide Multi-Context MSBB Results

In [9]:
# Load MSBB multi-context analysis results
msbb_multicon <- fread('/data/analysis_result/multi_context/MSBB/export/summary/MSBB.exported.toploci.bed.gz')

# Filter for significant results (credible set coverage > 0.95)
msbb_multicon <- msbb_multicon %>% filter(cs_coverage_0.95 > 0)
msbb_multicon %>% head()

# Identify multi-context results that overlap with AD GWAS variants
msbb_multicon_ad <- msbb_multicon %>% filter(variant_ID %in% ad_vars$variant_ID)
msbb_multicon_ad %>% dim()

# Extract gene IDs
tb5 <- msbb_multicon_ad %>% select(gene_ID) %>% distinct()

# Map gene IDs to gene reference data
tb5 <- gene_ref %>% 
  filter(gene_id %in% tb5$gene_ID) %>% 
  mutate(resources = 'multi_context', data_resource = 'FunGen_xQTL')

tb5 %>% head()

#chr,start,end,a1,a2,variant_ID,gene_ID,event_ID,cs_coverage_0.95,cs_coverage_0.7,cs_coverage_0.5,PIP,conditional_effect,lfsr
<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<chr>,<chr>
1,1023774,1023775,A,G,chr1:1023775:G:A,ENSG00000188157,BM_10_MSBB_eQTL;BM_22_MSBB_eQTL;BM_36_MSBB_eQTL;BM_44_MSBB_eQTL,1,1,1,0.97210539,0.104506519939872;0.125894884716484;0.0900918435608349;0.11841837243765,3.79924606022026e-12;3.84197473318569e-11;7.66144505134629e-11;1.4305998342331e-16
1,1185633,1185634,A,G,chr1:1185634:G:A,ENSG00000186891,BM_10_MSBB_eQTL;BM_22_MSBB_eQTL;BM_36_MSBB_eQTL;BM_44_MSBB_eQTL,1,1,1,0.14623895,0.634539886004572;0.51361551629155;0.591478449188189;0.582230552873455,6.89576902647313e-13;5.66680716108471e-09;2.71034501950873e-12;9.65056341704484e-13
1,1185960,1185961,T,C,chr1:1185961:C:T,ENSG00000186891,BM_10_MSBB_eQTL;BM_22_MSBB_eQTL;BM_36_MSBB_eQTL;BM_44_MSBB_eQTL,1,1,1,0.07711689,0.634115468678271;0.516653150656981;0.540331824203107;0.565234066186421,6.89576902647313e-13;5.66680716108471e-09;2.71034501950873e-12;9.65056341704484e-13
1,1186413,1186414,A,G,chr1:1186414:G:A,ENSG00000186891,BM_10_MSBB_eQTL;BM_22_MSBB_eQTL;BM_36_MSBB_eQTL;BM_44_MSBB_eQTL,1,1,1,0.26701266,0.641661665800936;0.523696513943397;0.580825918223139;0.582827363955203,6.89576902647313e-13;5.66680716108471e-09;2.71034501950873e-12;9.65056341704484e-13
1,1186643,1186644,C,T,chr1:1186644:T:C,ENSG00000186891,BM_10_MSBB_eQTL;BM_22_MSBB_eQTL;BM_36_MSBB_eQTL;BM_44_MSBB_eQTL,1,1,0,0.02118090,0.635804267352963;0.540146604599051;0.57339889758486;0.581141794657505,6.89576902647313e-13;5.66680716108471e-09;2.71034501950873e-12;9.65056341704484e-13
1,1186938,1186939,T,C,chr1:1186939:C:T,ENSG00000186891,BM_10_MSBB_eQTL;BM_22_MSBB_eQTL;BM_36_MSBB_eQTL;BM_44_MSBB_eQTL,1,1,0,0.02118090,0.635804267352963;0.540146604599051;0.57339889758486;0.581141794657505,6.89576902647313e-13;5.66680716108471e-09;2.71034501950873e-12;9.65056341704484e-13


[1] 101  14

#chr,start,end,gene_id,gene_name,resources,data_resource
<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
chr2,202871765,202871766,ENSG00000163596,ICA1L,multi_context,FunGen_xQTL
chr7,55019016,55019017,ENSG00000146648,EGFR,multi_context,FunGen_xQTL
chr12,113185525,113185526,ENSG00000139405,RITA1,multi_context,FunGen_xQTL
chr15,63276017,63276018,ENSG00000138613,APH1B,multi_context,FunGen_xQTL
chr17,44345261,44345262,ENSG00000030582,GRN,multi_context,FunGen_xQTL
chr20,44966478,44966479,ENSG00000101109,STK4,multi_context,FunGen_xQTL


## Data Source 6: updated ColocBoost list

In [44]:
# Load multi-gene analysis results from ROSMAP and MSBB datasets
cb_apr25 <- fread('/data/analysis_result//ColocBoost//export//summary//ad_xqtl_colocboost_export_filtered.bed.gz')

In [45]:
# Map gene IDs to gene reference data
tb6 <- gene_ref %>% 
  filter(gene_id %in% cb_apr25$gene_ID) %>% 
  mutate(resources = 'ColocBoost', data_resource = 'FunGen_xQTL')

tb6 %>% head()

#chr,start,end,gene_id,gene_name,resources,data_resource
<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
chr1,23980926,23980927,ENSG00000188529,SRSF10,ColocBoost,FunGen_xQTL
chr1,160285129,160285130,ENSG00000258465,AL139011.2,ColocBoost,FunGen_xQTL
chr1,161215233,161215234,ENSG00000158869,FCER1G,ColocBoost,FunGen_xQTL
chr1,207454229,207454230,ENSG00000117322,CR2,ColocBoost,FunGen_xQTL
chr1,207496146,207496147,ENSG00000203710,CR1,ColocBoost,FunGen_xQTL
chr1,231422286,231422287,ENSG00000135766,EGLN1,ColocBoost,FunGen_xQTL


In [46]:
tb1 %>% head

gene_id,resources,#chr,start,end,gene_name,data_resource
<chr>,<chr>,<int>,<int>,<int>,<chr>,<chr>
ENSG00000117322,"ColocBoost,single_context,MetaBrain",1,207454229,207454230,CR2,FunGen_xQTL
ENSG00000125944,ColocBoost,1,23344335,23344336,HNRNPR,FunGen_xQTL
ENSG00000135766,ColocBoost,1,231422286,231422287,EGLN1,FunGen_xQTL
ENSG00000142798,ColocBoost,1,21937309,21937310,HSPG2,FunGen_xQTL
ENSG00000143321,ColocBoost,1,156766924,156766925,HDGF,FunGen_xQTL
ENSG00000143771,ColocBoost,1,224356857,224356858,CNIH4,FunGen_xQTL


In [47]:
cb_gene_not_in_tb6 <- setdiff(tb1 %>% filter(str_detect(resources, 'ColocBoost')) %>% pull(gene_id), tb6$gene_id)
cb_gene_not_in_tb6 %>% length # those are genes removed from ColocBoost new version

[1] 28

In [48]:
cb_gene_not_in_tb1 <- setdiff(tb6$gene_id, tb1 %>% filter(str_detect(resources, 'ColocBoost')) %>% pull(gene_id))
cb_gene_not_in_tb1 %>% length # those are new gene from ColocBoost new version

[1] 75

remove the 28 CollocBoost genes not showing in tb6 from tb1 

In [49]:
nrow(tb1)
tb1 <- tb1 %>%
  mutate(
    resources = ifelse(gene_id %in% cb_gene_not_in_tb6, sub('ColocBoost', '', resources), resources),
  ) %>% filter(resources != '')
nrow(tb1)

[1] 193

[1] 168

In [50]:
193-168

[1] 25

removed 25 genes, 3 genes are kept because they are not contributed by ColocBoost only

## Data Source 7: TWAS Results

In [37]:
# Load twas results
twas <- fread('/data/analysis_result/twas/export/summary/FunGen_twas.exported.bed.gz')

In [55]:
twas %>% head
twas %>% dim
twas %>% pull(molecular_id) %>% unique %>% length

#chr,start,end,molecular_id,TADB_start,TADB_end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,923921,923922,ENSG00000187634,0,6480000,AC_DeJager_eQTL,Bellenguez_2022,bayes_l,TRUE,FALSE,0.06358278,4.713139e-10,-0.5516557,0.5811842,eQTL,chr1_16103_2888443
1,923921,923922,ENSG00000187634,0,6480000,AC_DeJager_eQTL,Bellenguez_2022,bayes_r,TRUE,TRUE,0.09006772,8.293727e-14,-0.6079598,0.5432142,eQTL,chr1_16103_2888443
1,923921,923922,ENSG00000187634,0,6480000,AC_DeJager_eQTL,Bellenguez_2022,enet,TRUE,FALSE,0.07363354,1.811244e-11,-0.8264034,0.4085753,eQTL,chr1_16103_2888443
1,923921,923922,ENSG00000187634,0,6480000,AC_DeJager_eQTL,Bellenguez_2022,lasso,TRUE,FALSE,0.07175097,3.341192e-11,-0.7787284,0.4361397,eQTL,chr1_16103_2888443
1,923921,923922,ENSG00000187634,0,6480000,AC_DeJager_eQTL,Bellenguez_2022,mrash,TRUE,FALSE,0.08157779,1.353045e-12,-0.1803822,0.8568525,eQTL,chr1_16103_2888443
1,923921,923922,ENSG00000187634,0,6480000,AC_DeJager_eQTL,Bellenguez_2022,mrmash,TRUE,FALSE,0.07746210,5.199122e-12,-1.4978502,0.1341722,eQTL,chr1_16103_2888443


[1] 9452198      17

[1] 16880

In [144]:
n_imputable_genes <- twas %>%
  filter(is_imputable) %>%                  # keep only imputable rows
  distinct(molecular_id, context, gwas_study) %>%         # count unique gene-context pairs
  nrow()
n_imputable_genes

[1] 1561127

In [145]:
# Compute cutoff using Bonferroni-style correction
cutoff <- 0.05 / n_imputable_genes
cutoff

[1] 3.202814e-08

In [146]:
twas %>% filter(twas_pval < cutoff) %>% pull( molecular_id) %>% unique %>% length

[1] 490

In [147]:
twas_sig <- twas %>%
  mutate(sig = twas_pval < cutoff) %>%
  group_by(molecular_id, context, gwas_study) %>%
  summarise(
    n_methods = n(),
    n_sig = sum(sig),
    best_sig = any(sig & is_selected_method),
    .groups = "drop"
  ) %>%
  mutate(pass = (n_sig / n_methods > 0.5) | best_sig) %>%
  mutate(pass_strict = (n_sig / n_methods > 0.5)) %>%
  filter(pass)


In [148]:
twas_sig %>% pull(molecular_id) %>% unique %>% length

[1] 305

In [149]:
# Extract gene IDs
tb7 <- twas_sig %>% select(molecular_id) %>% distinct()

# Map gene IDs to gene reference data
tb7 <- gene_ref %>% 
  filter(gene_id %in% tb7$molecular_id) %>% 
  mutate(resources = 'twas', data_resource = 'FunGen_xQTL')

tb7 %>% head()

#chr,start,end,gene_id,gene_name,resources,data_resource
<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
chr1,15976131,15976132,ENSG00000116809,ZBTB17,twas,FunGen_xQTL
chr1,26693235,26693236,ENSG00000117713,ARID1A,twas,FunGen_xQTL
chr1,42817396,42817397,ENSG00000177868,SVBP,twas,FunGen_xQTL
chr1,43389881,43389882,ENSG00000198198,SZT2,twas,FunGen_xQTL
chr1,43650148,43650149,ENSG00000284989,KDM4A,twas,FunGen_xQTL
chr1,43650148,43650149,ENSG00000284989,AL451062.2,twas,FunGen_xQTL


## Combining and Standardizing All Data Sources

In [195]:
# Combine all data sources
tb <- rbind(tb1, tb2, fill = TRUE) %>% 
  rbind(., tb3, fill = TRUE) %>%
  rbind(., tb4, fill = TRUE) %>%
  rbind(., tb5, fill = TRUE) %>%
  rbind(., tb6, fill = TRUE) 

# Standardize nomenclature across data sources
tb <- tb %>%
  mutate(
    data_resource = ifelse(data_resource == "PI_list", "FunGen_AD_PI", data_resource),
    resources = gsub("single_multicon", "single_multi_context", resources),
    data_resource = gsub("FunGen_ADSP", "ADSP_GVC", data_resource)
  )


## Creating the Final Gene List


# Create a clean, consolidated gene list
tb_clean <- tb %>% 
  group_by(gene_id) %>% 
  summarise(
    resources = paste(unique(na.omit(unlist(str_split(resources, ",")))), collapse = ","),
    data_resources = paste(unique(data_resource), collapse = ",")
  ) %>%        
  left_join(tb %>% select(-resources)) %>% 
  select(-data_resource) %>% 
  distinct(gene_id, .keep_all = TRUE)

# Rename columns for clarity
tb_clean <- tb_clean %>% 
  rename(
    'resource_in_FunGen_xQTL' = 'resources', 
    'chr' = '#chr'
  ) %>% mutate(chr = gsub('chr','',chr)) %>% 
  mutate(resource_in_FunGen_xQTL = ifelse(str_detect(resource_in_FunGen_xQTL, '^,'), sub(',','',resource_in_FunGen_xQTL), resource_in_FunGen_xQTL))

# Display sample records and summary statistics
tb_clean %>% head()

# Count unique genes in the final list
unique_gene_count <- tb_clean$gene_id %>% unique() %>% length()
cat("Total unique genes:", unique_gene_count, "\n")


Joining with `by = join_by(gene_id)`


gene_id,resource_in_FunGen_xQTL,data_resources,chr,start,end,gene_name
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>
ENSG00000002919,ColocBoost,FunGen_xQTL,17,48103356,48103357,SNX11
ENSG00000005100,ColocBoost,FunGen_xQTL,17,5468981,5468982,DHX33
ENSG00000006530,ColocBoost,FunGen_xQTL,7,141551277,141551278,AGK
ENSG00000008294,ColocBoost,FunGen_xQTL,17,51120867,51120868,SPAG9
ENSG00000013503,ColocBoost,FunGen_xQTL,12,106357747,106357748,POLR3B
ENSG00000015592,,FunGen_AD_PI,8,27258419,27258420,STMN4


Total unique genes: 287 


In [196]:
tb_clean %>% pull(data_resources) %>% unique

[1] "FunGen_xQTL"                       "FunGen_AD_PI"                     
[3] "ADSP_GVC,FunGen_AD_PI"             "ADSP_GVC"                         
[5] "FunGen_xQTL,ADSP_GVC"              "FunGen_xQTL,ADSP_GVC,FunGen_AD_PI"
[7] "FunGen_xQTL,FunGen_AD_PI"          "FunGen_AD_PI,FunGen_xQTL"

In [197]:
fwrite(tb_clean,"~/data/resource/AD_genes_FunGen_AD_GVC_xQTL_20250325.tsv", sep = '\t')

### twas table (optional)

In [200]:
# Combine all data sources
tb <- rbind(tb1, tb2, fill = TRUE) %>% 
  rbind(., tb3, fill = TRUE) %>%
  rbind(., tb4, fill = TRUE) %>%
  rbind(., tb5, fill = TRUE) %>%
  rbind(., tb6, fill = TRUE) %>%
  rbind(., tb7, fill = TRUE) 

# Standardize nomenclature across data sources
tb <- tb %>%
  mutate(
    data_resource = ifelse(data_resource == "PI_list", "FunGen_AD_PI", data_resource),
    resources = gsub("single_multicon", "single_multi_context", resources),
    data_resource = gsub("FunGen_ADSP", "ADSP_GVC", data_resource)
  )


## Creating the Final Gene List


# Create a clean, consolidated gene list
tb_clean <- tb %>% 
  group_by(gene_id) %>% 
  summarise(
    resources = paste(unique(na.omit(unlist(str_split(resources, ",")))), collapse = ","),
    data_resources = paste(unique(data_resource), collapse = ",")
  ) %>%        
  left_join(tb %>% select(-resources)) %>% 
  select(-data_resource) %>% 
  distinct(gene_id, .keep_all = TRUE)

# Rename columns for clarity
tb_clean <- tb_clean %>% 
  rename(
    'resource_in_FunGen_xQTL' = 'resources', 
    'chr' = '#chr'
  ) %>% mutate(chr = gsub('chr','',chr)) %>% 
  mutate(resource_in_FunGen_xQTL = ifelse(str_detect(resource_in_FunGen_xQTL, '^,'), sub(',','',resource_in_FunGen_xQTL), resource_in_FunGen_xQTL)) %>%
 filter(str_detect(resource_in_FunGen_xQTL, 'twas'))

# Display sample records and summary statistics
tb_clean %>% head()

# Count unique genes in the final list
unique_gene_count <- tb_clean$gene_id %>% unique() %>% length()
cat("Total unique genes:", unique_gene_count, "\n")


Joining with `by = join_by(gene_id)`


gene_id,resource_in_FunGen_xQTL,data_resources,chr,start,end,gene_name
<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>
ENSG00000001617,twas,FunGen_xQTL,3,50155044,50155045,SEMA3F
ENSG00000005469,twas,FunGen_xQTL,7,87345663,87345664,CROT
ENSG00000006062,twas,FunGen_xQTL,17,45317028,45317029,MAP3K14
ENSG00000006459,twas,FunGen_xQTL,7,140176982,140176983,KDM7A
ENSG00000006704,twas,FunGen_xQTL,7,74453789,74453790,GTF2IRD1
ENSG00000007171,twas,FunGen_xQTL,17,27800528,27800529,NOS2


Total unique genes: 305 


In [202]:
fwrite(tb_clean,"~/data/resource/AD_genes_FunGen_AD_twas_GVC_xQTL_20250325.tsv", sep = '\t')

## Summary of Results
The final gene list contains 243 genes incorporates evidence from multiple sources including FunGen xQTL analyses, ADSP database, and PI-curated lists. The integration of multi-gene and multi-context analyses provides a comprehensive view of genes potentially involved in Alzheimer's Disease pathology.



In [203]:
tb <- fread("~/data/resource/AD_genes_FunGen_AD_GVC_xQTL_20250325.tsv", sep = '\t')

In [204]:
dim(tb)

[1] 287   7